In [ ]:
!pip install ultralytics roboflow opencv-python matplotlib pillow

In [ ]:
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
from roboflow import Roboflow
import numpy as np
from PIL import Image
import os

In [ ]:
!pip install roboflow


def download_dataset():
    from roboflow import Roboflow
    rf = Roboflow(api_key="QHr9qKdWagsf1TvMIUq1")
    project = rf.workspace("sanjrani").project("potato-plant-30d")
    version = project.version(9)
    dataset = version.download("yolov11")
    return dataset.location

# Uncomment to download dataset
dataset_path = download_dataset()
print(f"Dataset downloaded to: {dataset_path}")

In [ ]:
# CELL 2.5: Check GPU first (run this before training)
def check_gpu_setup():
    import torch
    if torch.cuda.is_available():
        print(f"✅ GPU Available: {torch.cuda.get_device_name(0)}")
        print(f"✅ Memory: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.1f} GB")
        return True
    else:
        print("❌ GPU not available - check Kaggle accelerator settings")
        return False

check_gpu_setup()

In [ ]:
def train_model(dataset_path):
    import torch
    
    # Explicitly check and set device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"🚀 Training on: {device}")
    
    # Load model
    model = YOLO('yolo11m-seg.pt')  # Medium model for better accuracy
    
    # Train with explicit GPU settings
    results = model.train(
        data=f"{dataset_path}/data.yaml",
        epochs=10,  # Your original setting
        imgsz=640,
        batch=16,  # Increased for GPU
        device=device,  # Explicitly set device
        workers=2,  # Good for Kaggle
        name='potato_counter'
    )
    
    return "runs/segment/potato_counter3/weights/best.pt"

# Your training call
model_path = train_model("/kaggle/working/potato-plant-30D-9")
print(f"Model trained and saved to: {model_path}")

In [ ]:
from ultralytics import YOLO

In [ ]:
# Load your trained model (replace with your model path)
MODEL_PATH = '/kaggle/input/orthofiles/best.pt'  # Update this path
model = YOLO(MODEL_PATH)
print("Model loaded successfully!")

In [ ]:
import cv2
import matplotlib.pyplot as plt 
def count_potato_plants(image_path, confidence=0.5, show_image=True):
    """
    Count potato plants in an image
    
    Args:
        image_path: Path to the image
        confidence: Detection confidence threshold (0.0 - 1.0)
        show_image: Whether to display the annotated image
    
    Returns:
        int: Number of plants detected
    """
    
    # Read image
    image = cv2.imread(IMAGE_PATH)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Run inference
    results = model(image, conf=confidence)
    
    # Count detections
    plant_count = 0
    if results[0].boxes is not None:
        plant_count = len(results[0].boxes)
    
    # Show annotated image
    if show_image:
        # Get annotated image from YOLO
        annotated_image = results[0].plot()
        
        # Display using matplotlib
        plt.figure(figsize=(10, 5))
        plt.imshow(annotated_image)
        plt.title(f'Detected {plant_count} Potato Plants', fontsize=16)
        plt.axis('off')
        plt.show()
    
    print(f"🌱 Found {plant_count} potato plants!")
    return plant_count
IMAGE_PATH = "/kaggle/input/tif-files/DJI_20250624143217_0002_D.tif"

# Count plants with default settings
plant_count = count_potato_plants(IMAGE_PATH, confidence=0.5)

In [ ]:
import cv2
import matplotlib.pyplot as plt 
def count_potato_plants(image_path, confidence=0.5, show_image=True):
    """
    Count potato plants in an image
    
    Args:
        image_path: Path to the image
        confidence: Detection confidence threshold (0.0 - 1.0)
        show_image: Whether to display the annotated image
    
    Returns:
        int: Number of plants detected
    """
    
    # Read image
    image = cv2.imread(IMAGE_PATH)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Run inference
    results = model(image, conf=confidence)
    
    # Count detections
    plant_count = 0
    if results[0].boxes is not None:
        plant_count = len(results[0].boxes)
    
    # Show annotated image
    if show_image:
        # Get annotated image from YOLO
        annotated_image = results[0].plot()
        
        # Display using matplotlib
        plt.figure(figsize=(15, 10))
        plt.imshow(annotated_image)
        plt.title(f'Detected {plant_count} Potato Plants', fontsize=16)
        plt.axis('off')
        plt.show()
    
    print(f"🌱 Found {plant_count} potato plants!")
    return plant_count
IMAGE_PATH = "/kaggle/input/sampling-dataset/s3 blue hoop 4m.jpg"

# Count plants with default settings
plant_count = count_potato_plants(IMAGE_PATH, confidence=0.5)

In [ ]:
pip install ultralytics sahi opencv-python-headless


In [ ]:
import os
# Increase pixel limit before any OpenCV import
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = str(10**13)  # 1 trillion pixels

from PIL import Image
import numpy as np
import torch
from sahi import AutoDetectionModel

from sahi.predict import get_sliced_prediction
import cv2

# Load image with PIL (handles large files better)
image_path = '/kaggle/input/sampling-dataset/s3 rainbow hoop 3m.jpg'
pil_image = Image.open(image_path)
ortho_image = np.array(pil_image)  # Shape: (height, width, 3)

# Convert RGB to BGR for OpenCV compatibility
ortho_image = cv2.cvtColor(ortho_image, cv2.COLOR_RGB2BGR)

# Initialize model (corrected parameters)
detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path='/kaggle/input/potatot30d-weights/potato t30D.pt',
    confidence_threshold=0.2,
    device='cuda:0' if torch.cuda.is_available() else 'cpu',
    category_mapping={'0': 'potato T30 D'}  # Replace with your class mapping
)

# Perform tiled inference
result = get_sliced_prediction(
    ortho_image,  # Use the numpy array directly
    detection_model,
    slice_height=3000,
    slice_width=3000,
    overlap_height_ratio=0.2,
    overlap_width_ratio=0.2
)


In [ ]:
result.export_visuals('output_dir2/')  # Saves mask overlays
# result.to_coco('annotations.json')     # COCO format for GIS

In [ ]:
from IPython.display import Image, display

# Display composite visualization
display(Image(filename='/kaggle/working/output_dir2/prediction_visual.png'))

# Display sample mask
# display(Image(filename='output/instance_0_mask.png'))


In [ ]:
import os
# Set environment variables FIRST
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = str(10**13)  # 1 trillion pixels


In [1]:
# Install missing dependencies
!pip install -q torch sahi opencv-python-headless matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.0/112.0 kB 7.5 MB/s eta 0:00:00


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.1 MB/s eta 0:00:0000:01


In [ ]:
import os
# Set environment variables FIRST
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = str(10**13)  # 1 trillion pixels


from PIL import Image
import numpy as np
import torch
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
import cv2
import matplotlib.pyplot as plt

# Load and preprocess image
image_path = '/kaggle/input/lomond-files-t45d/DJI_20250625044135_0012_D.JPG'
pil_image = Image.open(image_path)
ortho_image = np.array(pil_image)
ortho_image = cv2.cvtColor(ortho_image, cv2.COLOR_RGB2BGR)

# Initialize model
detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path='/kaggle/input/potatot30d-weights/potato t30D.pt',
    confidence_threshold=0.1,
    device='cuda:0' if torch.cuda.is_available() else 'cpu',
    category_mapping={'0': 'potato'}
)

# Perform inference
result = get_sliced_prediction(
    ortho_image,
    detection_model,
    slice_height=3000,
    slice_width=3000,
    overlap_height_ratio=0.2,
    overlap_width_ratio=0.2
)

# Count plants
plant_count = len(result.object_prediction_list)
print(f"Detected plants: {plant_count}")

result.export_visuals('output_dir2/')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Performing prediction on 4 slices.
Detected plants: 85


In [ ]:
from IPython.display import Image, display

# Display composite visualization
display(Image(filename='/kaggle/working/output_dir2/prediction_visual.png'))


In [ ]:
def count_potato_plants(image_path, confidence, show_image=False):
    import os
    import cv2
    print(f"\n🔥 Processing NEW image: {image_path}")
    img = cv2.imread(image_path)
    print(f"Image shape: {img.shape} | Mean pixel value: {img.mean():.1f}")
    

    
    results = {}
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    
    # Get all image files (with absolute paths)
    image_files = []
    for ext in image_extensions:
        image_files.extend([
            os.path.join(image_folder, f) 
            for f in os.listdir(image_folder) 
            if f.lower().endswith(ext)
        ])
    
    print(f"Processing {len(image_files)} images...")
    
    total_plants = 0
    for i, image_path in enumerate(image_files, 1):
        try:
            # Verify the image is readable and unique
            img = cv2.imread(image_path)
            if img is None:
                print(f"⚠️ Could not read: {image_path}")
                results[os.path.basename(image_path)] = 0
                continue
                
            # DEBUG: Print image hash to verify uniqueness
            img_hash = hash(img.tobytes())
            print(f"Processing {i}/{len(image_files)}: {os.path.basename(image_path)} [Hash: {img_hash}]")
            
            # Count plants
            count = count_potato_plants(image_path, confidence, show_image=False)
            
            results[os.path.basename(image_path)] = count
            total_plants += count
            
            # Force cleanup
            del img
            
        except Exception as e:
            print(f"⚠️ Error processing {os.path.basename(image_path)}: {str(e)}")
            results[os.path.basename(image_path)] = 0
    
    print(f"\n📊 SUMMARY:")
    print(f"Total Images: {len(image_files)}")
    print(f"Total Plants: {total_plants}")
    print(f"Average Plants per Image: {total_plants/len(image_files):.1f}")
    
    return results